*본 코드는 "What is the secret behind lotto numbers?  \\- Reflections on the fairness of the lotto and winning numbers" 에 대한 파이썬 코드 파일임.

In [2]:
#패키지 import

import pandas as pd
import numpy as np
import itertools
from scipy import stats

### 2장 적합도 검정

#### 2.0.1 피어슨 카이제곱 통계량의 점근 근사이론을 이용한 검정

각 숫자 독립성 검정에서 카이제곱 통계량은
$X^2 \approx \frac{N-k}{N-1}\chi^2_{N-1}$



각 쌍 독립성 검정에서 카이제곱 통계량은
$X^2 ≈ ((k-1)\binom{N-3}{k-2}/\binom{N-2}{k-2})\chi^2_{N-1} + \binom{N-4}{k-2}/\binom{N-2}{k-2}\chi^2_{N(N-3)/2}$, in our case, N=45, k =7

In [3]:
#실제 데이터 분할표 만들기
real = pd.read_excel('korea_lottery_winners.xlsx', header = None)
real = real.iloc[:,:6]
# 각 숫자에 대한 분할표
freqtable1 = pd.Series(real.values.flatten()).value_counts().sort_index()

# 두 숫자의 쌍에 대한 분할표
numbers = list(range(1,46))
combinations = list(itertools.combinations(numbers, 2))

temp = []
for pair in combinations:
  temp.append(sum(all(element in draw for element in pair) for draw in real.values))

freqtable2 = pd.DataFrame(temp, index = combinations)

In [4]:
# 분할표의 카이제곱 통계량 계산하는 함수.
def chisq(table):
      # expected 값은 table 내의 모든 값의 합을 table의 행 개수로 나눈 값
      expected = table.sum() / table.shape[0]
      # chisq 계산
      return ((table - expected) ** 2 / expected).sum()


In [5]:
# Ho에 대한 점근적 카이제곱 검정,

N = 45
k = 6

# #moment-matching 시키면(pair 검정통계량)

# 점근적 독립성 검정

w1 = (N-k)*(k-1)/(N-2)
w2 = ((N-k)*(N-k-1))/((N-2)*(N-3))
w = np.array([w1, w2])

v1 = N-1
v2 = N*(N-3)/2
v = np.array([v1,v2])


b = (((w**3)*v).sum()) / (((w**2)*v).sum())
V = ((((w**2)*v).sum())**3) / ((((w**3)*v).sum())**2)
a = (w*v).sum() - b*V


X2_1 = chisq(freqtable1)
X2_2 = chisq(freqtable2)

print("각 숫자에 대한 점근적 검정 p-value:", stats.chi2.sf(X2_1*(N-1)/(N-k), N-1))
print("각 쌍에 대한 점근적 검정 p-value:",stats.chi2.sf((X2_2 - a)/b, V))


각 숫자에 대한 점근적 검정 p-value: 0.8687567715428611
각 쌍에 대한 점근적 검정 p-value: [0.80196892]


#### 2.1 Bootstrap 검정

In [6]:
# Ho에 대한 Bootstrap 검정.

# D회차간의 랜덤한 로또번호를 generate 시키자.
#trial generate 시키는 함수
def trial(D):
    return [sorted(np.random.choice(np.arange(1, 46), size=6, replace=False)) for _ in range(D)]

#숫자 분할표 생성함수
def gen_freqtable1(data):
  flattened_data = np.concatenate(data)
  lst = pd.Series(flattened_data).value_counts().sort_index()
  return lst

#쌍 분할표 생성함수
def gen_freqtable2(data):
  combinations = list(itertools.combinations(list(range(1,46)), 2))
  # 45x45 매트릭스 생성
  frequency_matrix = np.zeros((45, 45))

  for draw in data:
      indices = np.array(draw) - 1  # 숫자 1~45를 인덱스 0~44로 조정
      for i, j in itertools.combinations(indices, 2):
          frequency_matrix[i, j] += 1
          frequency_matrix[j, i] += 1

  upper_triangular_indices = np.triu_indices_from(frequency_matrix, k=1)
  freq_1d_array = frequency_matrix[upper_triangular_indices]

  return freq_1d_array

In [7]:
# K회 생성된 로또 샘플에 대해 기존 chi-square 통계량으로 p-value를 계산하자.
# 분할표의 카이제곱 통계량 계산하는 함수.
np.random.seed(777)

D = 1131
K = 1000 # bootstrap K번 반복
def generate_chisq_values(gen_freqtable, trial_func, D, K):
    # 결과를 미리 numpy 배열로 생성해둠
    X2values = np.empty(K)
    for i in range(K):
        table = gen_freqtable(trial_func(D))
        X2values[i] = chisq(table)
    return X2values

chisq_values_single = generate_chisq_values(gen_freqtable1, trial, D, K)
chisq_values_pair = generate_chisq_values(gen_freqtable2, trial, D, K)


In [8]:
#실제 데이터의 검정통계량 값을 이용하여 계산한 p-value
print(np.mean(int(chisq(freqtable1)) <= np.array(chisq_values_single)))
print(np.mean(int(chisq(freqtable2)) <= np.array(chisq_values_pair)))

0.898
0.785


C:\Users\jmjwj\AppData\Local\Temp\ipykernel_32428\28812861.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(np.mean(int(chisq(freqtable2)) <= np.array(chisq_values_pair)))
